In [1]:
pip install mediapipe

Note: you may need to restart the kernel to use updated packages.


In [9]:
import mediapipe as mp
import cv2
import numpy as np
import uuid
import os
from tensorflow.keras.models import load_model
from pathlib import Path
from glob import glob


In [10]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [18]:
MODEL_DIR = './aslalphabetcnnmodel1'
MODEL_PATH = MODEL_DIR + '/ann-model98.h5'
MODEL_WEIGHTS_PATH = MODEL_DIR + '/ann-model98.weights.h5'
CLASSES = ['A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'space',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z']

In [19]:
# def preprocess_image(image):
#     sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=5)
#     return sobely

In [20]:
def load_model_from_disk():
    '''A convenience method for re-running certain parts of the
    analysis locally without refitting all the data.'''
    model_file = Path(MODEL_PATH)
    model_weights_file = Path(MODEL_WEIGHTS_PATH)
                      
    if model_file.is_file() and model_weights_file.is_file():
        print('Retrieving model from disk...')
        model = load_model(model_file.__str__())
                      
        print('Loading CNN model weights from disk...')
        model.load_weights(model_weights_file)
        return model
    
    return None

signDetector = load_model_from_disk()

Retrieving model from disk...
Loading CNN model weights from disk...


In [30]:
cap = cv2.VideoCapture(0)
_, frame = cap.read()
h, w, c = frame.shape

i = 0
text = ""

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5,max_num_hands=4) as hands: 
    while cap.isOpened():
        i = i + 1
        ret, frame = cap.read()
        
        # BGR 2 RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Flip on horizontal
        image = cv2.flip(image, 1)
        
        # Set flag
        image.flags.writeable = False
        
        # Detections
        croppedImage = cv2.resize(image,(200, 200))
        results = hands.process(croppedImage)
        
        # Set flag to true
        image.flags.writeable = True
        
        # RGB 2 BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        font = cv2.FONT_HERSHEY_SIMPLEX
  
        # org
        org = (50, 50)

        # fontScale
        fontScale = 1

        # Blue color in BGR
        color = (255, 0, 0)

        # Line thickness of 2 px
        thickness = 2
        
        # Detections
        # print(results)
        
        x = []
        # Rendering results
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
#                 x_max = 0
#                 y_max = 0
#                 x_min = w
#                 y_min = h
                landmarksForImage = [[lm.x,lm.y,lm.z] for lm in hand.landmark]   
#                     x, y = int(lm.x * w), int(lm.y * h)
#                     if x > x_max:
#                         x_max = x
#                     if x < x_min:
#                         x_min = x
#                     if y > y_max:
#                         y_max = y
#                     if y < y_min:
#                         y_min = y
#                 cv2.rectangle(image, (x_min - 25, y_min - 25), (x_max + 25, y_max + 25), (0, 255, 0), 2)
#                 img1 = image[x_min:x_max,y_min:y_max]               
#                 mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
#                                         mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
#                                         mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),
#                                        )
            x.append(landmarksForImage)
            if i % 40 == 0 :
#                 img = preprocess_image(img1)
#                 croppedImage = cv2.resize(img,(64, 64))
#                 prediction = signDetector.predict(x.reshape(-1, 64, 64, 3))
                prediction = signDetector.predict(x)
                r = np.argmax(prediction, axis=1)
                text = text + CLASSES[r[0]]
        cv2.putText(image,text,org, font, fontScale, color, thickness, cv2.LINE_AA)
        cv2.imshow('Hand Tracking', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()